In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import os
from tensorflow.keras import layers
import string

In [17]:
BATCH_SIZE = 128
NUM_HEADS = 8
NUM_BLOCKS = 2
EMBED_DIM = 256
DENSE_DIM = 1024
DROPOUT_RATE = 0.3
CHUNK_LENGTH = 256

In [18]:
df = pd.read_csv('/kaggle/input/the-bards-best-a-character-modeling-dataset/train.csv')

In [19]:
text = df.values[0][0]
text = re.sub(r'\s+', ' ', str(text)).strip()

In [20]:
import re

words = text.split()

# Count unique words
unique_words = set(words)
print(f"Total words: {len(words)}")
print(f"Unique words: {len(unique_words)}")


Total words: 182499
Unique words: 23841


In [21]:
def chunk_text_by_words(text, max_words, stride=None):
    words = text.split()
    if stride is None:
        stride = max_words // 2
    chunks = []
    for i in range(0, len(words) - max_words, stride):
        chunk = ' '.join(words[i:i + max_words])
        chunks.append(chunk)
    return chunks

In [22]:
chunks = chunk_text_by_words(text, CHUNK_LENGTH+1, 30)

In [23]:
print(np.shape(chunks))

(6075,)


In [24]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 12300
sequence_length = CHUNK_LENGTH+1

vectorizer = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=custom_standardization
)

vectorizer.adapt(chunks)

In [25]:
vocab = vectorizer.get_vocabulary()
print("Total unique tokens in vocabulary:", len(vocab))

Total unique tokens in vocabulary: 12074


In [26]:
def make_dataset(chunks):
    tokens = vectorizer(chunks)
    tokens_inp = tokens[:,:CHUNK_LENGTH]
    tokens_out = tokens[:,1:]
    ds = tf.data.Dataset.from_tensor_slices((tokens_inp,tokens_out))
    ds = ds.batch(BATCH_SIZE)
    ds = ds.shuffle(1024).prefetch(16).cache()
    return ds

In [27]:
ds = make_dataset(chunks)

In [28]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, output_dim):
        super().__init__()
        self.positional_embedding = tf.keras.layers.Embedding(input_dim = sequence_length, output_dim = output_dim, mask_zero=False)
        self.token_embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim= output_dim, mask_zero=True)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.positional_embedding(positions)
        return embedded_tokens + embedded_positions

In [29]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, num_heads, embed_dim, dense_dim, dropout_rate):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                           key_dim=embed_dim//num_heads)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dense_proj = tf.keras.models.Sequential([
            tf.keras.layers.Dense(dense_dim, activation='relu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    def call(self, inputs):
        attn_out = self.attention(query=inputs,
                            key=inputs,
                            value=inputs,
                            use_causal_mask=True)
        norm1_out = self.layernorm1(attn_out+inputs)
        drop1_out = self.dropout1(norm1_out)
        dense_proj_out = self.dense_proj(drop1_out)
        norm2_out = self.layernorm2(drop1_out+dense_proj_out)
        drop2_out = self.dropout2(norm2_out)
        return drop2_out

In [30]:
inputs = tf.keras.layers.Input(shape=(None,))
embeddings = PositionalEmbedding(sequence_length, vocab_size, EMBED_DIM)(inputs)
x = embeddings
for layer in range(NUM_BLOCKS):
    x = TransformerDecoder(NUM_HEADS, EMBED_DIM, DENSE_DIM, DROPOUT_RATE)(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(vocab_size, activation='linear', kernel_initializer='glorot_uniform')(x)
transformer = tf.keras.models.Model(inputs, output)

In [31]:
transformer.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding_1          │ (None, None, 256)      │     3,214,592 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_1           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_2           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, None, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, None, 12300)    │     3,161,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,955,212 (30.35 MB)

 Trainable params: 7,955,212 (30.35 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
transformer.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding_1          │ (None, None, 256)      │     3,214,592 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_1           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_2           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, None, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, None, 12300)    │     3,161,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,955,212 (30.35 MB)

 Trainable params: 7,955,212 (30.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow.keras.backend as K

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=5e-4)

def perplexity(y_true, y_pred):
    cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
    return K.exp(K.mean(cross_entropy))

transformer.compile(loss = loss_fn,
                    metrics = ['accuracy', perplexity],
                    optimizer=opt)
transformer.fit(ds, epochs = 500)

In [41]:
import numpy as np
import tensorflow as tf

def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-9) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)
    
def generate_text(prompt, max_length=50):
    for _ in range(max_length):
        tokenized = vectorizer([prompt])
        preds = transformer(tokenized)
        next_id = tf.argmax(preds[0, -1, :]).numpy()
        next_word = vectorizer.get_vocabulary()[next_id]
        prompt += " " + next_word
        if next_id == 0:
            break
    return prompt

print(generate_text('thou shall'))

thou shall camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo camillo
